In [1]:
import numpy as np
import pandas as pd
from countryinfo import CountryInfo
import country_converter as coco
from geopy.geocoders import Nominatim
import json
from math import sin, cos, sqrt, atan2, radians
import pycountry
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import wbdata

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

C:\Users\annan\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [15]:
df = pd.read_csv('qog_std_cs_jan23.csv')
asylum = pd.read_csv('glob_data_final.csv')

<ipython-input-15-957ea1cde474>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  asylum = pd.read_csv('Data_for_reg_with0.csv')


In [18]:
asylum = asylum[~asylum['citizen'].isin([1,'1'])]

In [22]:
convert_ISO_3166_2_to_1 = {
'AF':'AFG',
'AX':'ALA',
'AL':'ALB',
'DZ':'DZA',
'AS':'ASM',
'AD':'AND',
'AO':'AGO',
'AI':'AIA',
'AQ':'ATA',
'AG':'ATG',
'AR':'ARG',
'AM':'ARM',
'AW':'ABW',
'AU':'AUS',
'AT':'AUT',
'AZ':'AZE',
'BS':'BHS',
'BH':'BHR',
'BD':'BGD',
'BB':'BRB',
'BY':'BLR',
'BE':'BEL',
'BZ':'BLZ',
'BJ':'BEN',
'BM':'BMU',
'BT':'BTN',
'BO':'BOL',
'BA':'BIH',
'BW':'BWA',
'BV':'BVT',
'BR':'BRA',
'IO':'IOT',
'BN':'BRN',
'BG':'BGR',
'BF':'BFA',
'BI':'BDI',
'KH':'KHM',
'CM':'CMR',
'CA':'CAN',
'CV':'CPV',
'KY':'CYM',
'CF':'CAF',
'TD':'TCD',
'CL':'CHL',
'CN':'CHN',
'CX':'CXR',
'CC':'CCK',
'CO':'COL',
'KM':'COM',
'CG':'COG',
'CD':'COD',
'CK':'COK',
'CR':'CRI',
'CI':'CIV',
'HR':'HRV',
'CU':'CUB',
'CY':'CYP',
'CZ':'CZE',
'DK':'DNK',
'DJ':'DJI',
'DM':'DMA',
'DO':'DOM',
'EC':'ECU',
'EG':'EGY',
'SV':'SLV',
'GQ':'GNQ',
'ER':'ERI',
'EE':'EST',
'ET':'ETH',
'FK':'FLK',
'FO':'FRO',
'FJ':'FJI',
'FI':'FIN',
'FR':'FRA',
'GF':'GUF',
'PF':'PYF',
'TF':'ATF',
'GA':'GAB',
'GM':'GMB',
'GE':'GEO',
'DE':'DEU',
'GH':'GHA',
'GI':'GIB',
'GR':'GRC',
'GL':'GRL',
'GD':'GRD',
'GP':'GLP',
'GU':'GUM',
'GT':'GTM',
'GG':'GGY',
'GN':'GIN',
'GW':'GNB',
'GY':'GUY',
'HT':'HTI',
'HM':'HMD',
'VA':'VAT',
'HN':'HND',
'HK':'HKG',
'HU':'HUN',
'IS':'ISL',
'IN':'IND',
'ID':'IDN',
'IR':'IRN',
'IQ':'IRQ',
'IE':'IRL',
'IM':'IMN',
'IL':'ISR',
'IT':'ITA',
'JM':'JAM',
'JP':'JPN',
'JE':'JEY',
'JO':'JOR',
'KZ':'KAZ',
'KE':'KEN',
'KI':'KIR',
'KP':'PRK',
'KR':'KOR',
'KW':'KWT',
'KG':'KGZ',
'LA':'LAO',
'LV':'LVA',
'LB':'LBN',
'LS':'LSO',
'LR':'LBR',
'LY':'LBY',
'LI':'LIE',
'LT':'LTU',
'LU':'LUX',
'MO':'MAC',
'MK':'MKD',
'MG':'MDG',
'MW':'MWI',
'MY':'MYS',
'MV':'MDV',
'ML':'MLI',
'MT':'MLT',
'MH':'MHL',
'MQ':'MTQ',
'MR':'MRT',
'MU':'MUS',
'YT':'MYT',
'MX':'MEX',
'FM':'FSM',
'MD':'MDA',
'MC':'MCO',
'MN':'MNG',
'ME':'MNE',
'MS':'MSR',
'MA':'MAR',
'MZ':'MOZ',
'MM':'MMR',
'NA':'NAM',
'NR':'NRU',
'NP':'NPL',
'NL':'NLD',
'AN':'ANT',
'NC':'NCL',
'NZ':'NZL',
'NI':'NIC',
'NE':'NER',
'NG':'NGA',
'NU':'NIU',
'NF':'NFK',
'MP':'MNP',
'NO':'NOR',
'OM':'OMN',
'PK':'PAK',
'PW':'PLW',
'PS':'PSE',
'PA':'PAN',
'PG':'PNG',
'PY':'PRY',
'PE':'PER',
'PH':'PHL',
'PN':'PCN',
'PL':'POL',
'PT':'PRT',
'PR':'PRI',
'QA':'QAT',
'RE':'REU',
'RO':'ROU',
'RU':'RUS',
'RW':'RWA',
'BL':'BLM',
'SH':'SHN',
'KN':'KNA',
'LC':'LCA',
'MF':'MAF',
'PM':'SPM',
'VC':'VCT',
'WS':'WSM',
'SM':'SMR',
'ST':'STP',
'SA':'SAU',
'SN':'SEN',
'RS':'SRB',
'SC':'SYC',
'SL':'SLE',
'SG':'SGP',
'SK':'SVK',
'SI':'SVN',
'SB':'SLB',
'SO':'SOM',
'ZA':'ZAF',
'GS':'SGS',
'ES':'ESP',
'LK':'LKA',
'SD':'SDN',
'SR':'SUR',
'SJ':'SJM',
'SZ':'SWZ',
'SE':'SWE',
'CH':'CHE',
'SY':'SYR',
'TW':'TWN',
'TJ':'TJK',
'TZ':'TZA',
'TH':'THA',
'TL':'TLS',
'TG':'TGO',
'TK':'TKL',
'TO':'TON',
'TT':'TTO',
'TN':'TUN',
'TR':'TUR',
'TM':'TKM',
'TC':'TCA',
'TV':'TUV',
'UG':'UGA',
'UA':'UKR',
'AE':'ARE',
'GB':'GBR',
'US':'USA',
'UM':'UMI',
'UY':'URY',
'UZ':'UZB',
'VU':'VUT',
'VE':'VEN',
'VN':'VNM',
'VG':'VGB',
'VI':'VIR',
'WF':'WLF',
'EH':'ESH',
'YE':'YEM',
'ZM':'ZMB',
'ZW':'ZWE'
}

In [23]:
asylum =asylum.replace({"citizen":convert_ISO_3166_2_to_1}).replace({"geo":convert_ISO_3166_2_to_1})

In [5]:
#asylum.to_csv('asylum.csv')

### Capitale des pays

In [24]:
pays = pd.read_csv('pays.csv',sep=";",encoding = 'ISO-8859-1')
pays = pays[['Capital', 'ISO Code (3 letters)']]
pays['Capital'] = pays['Capital'].replace('Yaound','Yaounde').replace("Copenagen","Copenhagen").replace("Reykjavk","Reykjavik")

df = pd.merge(df,pays,how='left',left_on=['ccodealp'],right_on=['ISO Code (3 letters)'])
df = df.drop(['ISO Code (3 letters)'], axis=1)


# On obtient les coordonnées spatiales des capitales
geolocator = Nominatim(user_agent="Your_Name")
df['capitale_lat'] = df.Capital.apply(lambda x: geolocator.geocode(x).latitude)
df['capitale_long'] = df.Capital.apply(lambda x: geolocator.geocode(x).longitude)

In [25]:
asylum = asylum.drop(['DATAFLOW','LAST UPDATE','freq','unit','asyl_app'],axis=1)

In [27]:
asylum = asylum.drop(['Unnamed: 0'],axis=1)

In [28]:
qog_dim = df[['ccodealp','capitale_lat','capitale_long']]
asylum = pd.merge(asylum, qog_dim, how='left', left_on=['geo'], right_on=['ccodealp'])

asylum = asylum.rename(columns={'capitale_lat':'capitale_lat_dest','capitale_long':'capitale_long_dest','ccodealp':'ccodealp_dest'})

In [31]:
asylum.citizen.unique()

array(['AND', 'ARE', 'AFG', 'ATG', 'ALB', 'ARM', 'AGO', 'ARG', 'AUT',
       'AUS', 'AZE', 'BIH', 'BRB', 'BGD', 'BEL', 'BFA', 'BGR', 'BHR',
       'BDI', 'BEN', 'BRN', 'BOL', 'BRA', 'BHS', 'BTN', 'BWA', 'BLR',
       'BLZ', 'CAN', 'COD', 'CAF', 'COG', 'CHE', 'CIV', 'COK', 'CHL',
       'CMR', 'CHN', 'COL', 'CRI', 'CUB', 'CPV', 'CYP', 'CZE', 'DEU',
       'DJI', 'DNK', 'DMA', 'DOM', 'DZA', 'ECU', 'EST', 'EGY', 'ESH',
       'GRC', 'ERI', 'ESP', 'ETH', 'FIN', 'FJI', 'FSM', 'FRA', 'GAB',
       'GRD', 'GEO', 'GHA', 'GMB', 'GIN', 'GNQ', 'GTM', 'GNB', 'GUY',
       'HND', 'HRV', 'HTI', 'HUN', 'IDN', 'IRL', 'ISR', 'IND', 'IRQ',
       'IRN', 'ISL', 'ITA', 'JAM', 'JOR', 'JPN', 'KEN', 'KGZ', 'KHM',
       'KIR', 'COM', 'KNA', 'PRK', 'KOR', 'KWT', 'KAZ', 'LAO', 'LBN',
       'LCA', 'LIE', 'LKA', 'LBR', 'LSO', 'LTU', 'LUX', 'LVA', 'LBY',
       'MAR', 'MCO', 'MDA', 'MNE', 'MDG', 'MHL', 'MKD', 'MLI', 'MMR',
       'MNG', 'MRT', 'MLT', 'MUS', 'MDV', 'MWI', 'MEX', 'MYS', 'MOZ',
       'NER', 'NGA',

In [30]:
asylum = asylum = pd.merge(asylum, qog_dim, how='left', left_on=['citizen'], right_on=['ccodealp'])
asylum = asylum.rename(columns={'capitale_lat':'capitale_lat_or','capitale_long':'capitale_long_or','ccodealp':'ccodealp_or'})

In [32]:
# distance entre deux coordoonées
def distance(lat1,lat2,lon1,lon2) :
    R = 6373.0
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance
    

In [33]:
# distance capitales
asylum['dist_capit'] =  asylum.apply(lambda x: distance(x.capitale_lat_or,x.capitale_lat_dest,x.capitale_long_or,x.capitale_long_dest), axis=1)

In [34]:
asylum = asylum.drop(['capitale_lat_dest','capitale_long_dest','capitale_lat_or','capitale_long_or'],axis=1)

###### Religion

In [36]:
religion = pd.read_csv('religion.csv').drop(['Unnamed: 0'],axis=1)

In [37]:
a = list(religion[religion['country_code']=="Unknown code"]['Country'])

a[0] = 'Congo (the Democratic Republic of the)'
a[1] = 'Congo (the)'
a[2] = 'Sao Tome and Principe'
a[3] = 'Tanzania, the United Republic of'
a[4] = 'Cabo Verde'
a[5] = 'Gambia (the)'
a[6] = "Côte d'Ivoire"
a[11] = "Bolivia (Plurinational State of)"
a[13] = 'Venezuela (Bolivarian Republic of)'
a[14] = 'China'
a[16] = 'Taiwan (Province of China)'
a[18] = 'Korea (the Republic of)'
a[17] =  "Korea (the Democratic People's Republic of)"
a[19] = 'Brunei Darussalam'
a[20] = "Lao People's Democratic Republic (the)"
a[21] = 'Viet Nam'
a[22] = 'India'
a[23] = 'Iran (Islamic Republic of)'
a[25] = 'Syrian Arab Republic (the)'
a[26] = 'Czechia'
a[27] = 'Russian Federation (the)'
a[29] = 'Moldova (the Republic of)'

In [38]:
religion1 = religion[religion['country_code']=="Unknown code"].reset_index()
religion2 = religion[religion['country_code']!="Unknown code"].reset_index()

In [39]:
religion1['Country'] = a

In [40]:
religion1 = religion1.drop(['index'],axis = 1)
religion2 = religion2.drop(['index'],axis = 1)

In [41]:
qog = pd.read_csv('qog_std_cs_jan23.csv')

In [42]:
df1 = qog.merge(religion2[['country_code','first_rel']],left_on='ccodealp',right_on='country_code')

In [43]:
df2 = qog.merge(religion1[['Country','first_rel']],left_on="cname",right_on="Country",how='inner')


In [44]:
df = pd.concat([df1,df2])

In [45]:
df = df.drop(['Country'],axis=1)

In [46]:
a = list(df['cname'])
b = list(qog['cname'])

df3 = qog[qog['cname'].isin(list(set(a).symmetric_difference(b)))]
df3['first_rel'] = ['Christian','Christian','Christian','Christian','Christian']

qog = pd.concat([df,df3])

<ipython-input-46-e7f8434a5e21>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['first_rel'] = ['Christian','Christian','Christian','Christian','Christian']


In [47]:
asylum['religion_or'] = asylum[['ccodealp_or']].merge(qog[['ccodealp','first_rel']],left_on = 'ccodealp_or',right_on='ccodealp',how='left')['first_rel']
asylum['religion_dest'] = asylum[['ccodealp_dest']].merge(qog[['ccodealp','first_rel']],left_on = 'ccodealp_dest',right_on='ccodealp',how='left')['first_rel']

In [48]:
conditions = [
    (asylum['religion_or'] == asylum['religion_dest']),
    (asylum['religion_or'] != asylum['religion_dest'])]

choices = ['Oui', 'Non']
asylum['religion_commune'] = np.select(conditions, choices)

#### Variable du QOG

In [27]:
# asylum_2018 = asylum[asylum['TIME_PERIOD']==2018][['citizen','geo','OBS_VALUE']]
# qog_2018 = qog[qog['year']==2018].drop(['cname','ccode','year','cname_year','ccodealp_year','ccodecow','version'],axis=1)

# qog_2018 = qog_2018[qog_2018.ccodealp.isin(list(asylum_2018.citizen))]

# # On supprime les colonnes qui ont plus de 30 na
# qog_2018 = qog_2018.dropna(thresh=len(qog_2018) - 30, axis=1)

# # On supprime à la main, les variables qui ne semblent pas adaptée
# qog_2018 = qog_2018[qog_2018.columns.drop(list(qog_2018.filter(regex='wdi_')))]
# qog_2018 = qog_2018[qog_2018.columns.drop(list(qog_2018.filter(regex='top_top')))]
# qog_2018 = qog_2018[qog_2018.columns.drop(list(qog_2018.filter(regex='vdem_')))]
# qog_2018 = qog_2018.drop(['pwt_pop','rd_inw','rd_outw','undp_hdi','van_index','atop_ally','bmr_demdur','mad_gdppc',
#                          'p_durable','p_polity2','cspf_sfi'],axis=1)


# # Imputation with KNN

# KNNImputer(missing_values=np.nan, n_neighbors=5, weights='uniform', metric='nan_euclidean')

# qog_2018_knn = qog_2018.drop(['ccode_qog','cname_qog','ccodealp'],axis=1)

# scaler = MinMaxScaler(feature_range=(0, 1))
# qog_2018_knn = pd.DataFrame(scaler.fit_transform(qog_2018_knn), columns = qog_2018_knn.columns)

# # Define KNN imputer and fill missing values
# knn_imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
# qog_2018_knn_imputed = pd.DataFrame(knn_imputer.fit_transform(qog_2018_knn), columns=qog_2018_knn.columns)

# qog_2018_knn_imputed['cname_qog'] = list(qog_2018['cname_qog'])
# qog_2018_knn_imputed['ccodealp'] = list(qog_2018['ccodealp'])

# asylum_2018 = asylum_2018[asylum_2018['OBS_VALUE']!=0]


# # Pearson for feature selection for origin country

# df = asylum_2018[['citizen','OBS_VALUE']].groupby('citizen').sum().merge(qog_2018_knn_imputed, left_on='citizen', right_on='ccodealp').drop(['cname_qog'],axis=1)

# def cor_selector(X, y,num_feats):
#     cor_list = []
#     feature_name = X.columns.tolist()
#     # calculate the correlation with y for each feature
#     for i in X.columns.tolist():
#         cor = np.corrcoef(X[i], y)[0, 1]
#         cor_list.append(cor)
#     # replace NaN with 0
#     cor_list = [0 if np.isnan(i) else i for i in cor_list]
#     # feature name
#     cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
#     # feature selection? 0 for not select, 1 for select
#     cor_support = [True if i in cor_feature else False for i in feature_name]
#     return cor_support, cor_feature

# X = df.iloc[:,1:].drop('ccodealp',axis=1)
# y = df.iloc[:,0]

# cor_support, cor_feature = cor_selector(X, y,10)

# print(str(len(cor_feature)), 'selected features',cor_feature)

# origine_qog = cor_feature
# origine_qog.append('ccodealp')

# origine_qog = qog_2018_knn_imputed[origine_qog]

# origine_qog = origine_qog.add_suffix('_origine')

# # Pearson for feature selection for destination country

# df = asylum_2018[['geo','OBS_VALUE']].groupby('geo').sum().merge(qog_2018_knn_imputed, left_on='geo', right_on='ccodealp').drop(['cname_qog'],axis=1)

# def cor_selector(X, y,num_feats):
#     cor_list = []
#     feature_name = X.columns.tolist()
#     # calculate the correlation with y for each feature
#     for i in X.columns.tolist():
#         cor = np.corrcoef(X[i], y)[0, 1]
#         cor_list.append(cor)
#     # replace NaN with 0
#     cor_list = [0 if np.isnan(i) else i for i in cor_list]
#     # feature name
#     cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
#     # feature selection? 0 for not select, 1 for select
#     cor_support = [True if i in cor_feature else False for i in feature_name]
#     return cor_support, cor_feature

# X = df.iloc[:,1:].drop('ccodealp',axis=1)
# y = df.iloc[:,0]

# cor_support, cor_feature = cor_selector(X, y,10)

# print(str(len(cor_feature)), 'selected features',cor_feature)
# cor_feature.remove('ht_region')


In [51]:
qog = pd.read_csv('qog_bas_ts_jan23.csv')

In [52]:
qog = qog[qog['year'].isin([2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021])]

In [53]:
# Variables origine qui ont été sélectionnées plus haut
origine = ['fh_status', 'fh_feb','wbgi_cce', 'fh_rol', 'fh_cl', 'fh_pair', 'wbgi_rle', 'wbgi_pve']
#Variables dest qui ont été sélectionnées plus haut
dest = [ 'fh_ep', 'fh_pr', 'ipu_l_sw', 'fh_fog', 'dr_ig',  'ht_region', 'wbgi_pve']
or_dest = list(set(origine+dest))
var = ['ccodealp','year'] + or_dest
qog = qog[var]

In [54]:
origine = list(asylum['citizen'].unique())
dest = list(asylum['geo'].unique())
pays = list(set(origine+dest))

In [55]:
qog = qog[qog['ccodealp'].isin(pays)]

In [56]:
# Imputation with KNN

KNNImputer(missing_values=np.nan, n_neighbors=5, weights='uniform', metric='nan_euclidean')

qog_knn = qog.drop(['ccodealp','year'],axis=1)

scaler = MinMaxScaler(feature_range=(0, 1))
qog_knn = pd.DataFrame(scaler.fit_transform(qog_knn), columns = qog_knn.columns)

# Define KNN imputer and fill missing values
knn_imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
qog_knn_imputed = pd.DataFrame(knn_imputer.fit_transform(qog_knn), columns=qog_knn.columns)

qog_knn_imputed['ccodealp'] = list(qog['ccodealp'])
qog_knn_imputed['year'] = list(qog['year'])

In [57]:
qog_knn_imputed_origine = qog_knn_imputed[['ccodealp','year','fh_status', 'fh_feb','wbgi_cce', 'fh_rol', 'fh_cl', 'fh_pair', 'wbgi_rle', 'wbgi_pve']]
qog_knn_imputed_dest = qog_knn_imputed[['ccodealp','year','fh_ep', 'fh_pr', 'ipu_l_sw', 'fh_fog', 'dr_ig',  'ht_region', 'wbgi_pve']]

In [58]:
qog_knn_imputed_origine = qog_knn_imputed_origine.add_suffix('_origine').rename(columns={'ccodealp_origine':'ccodealp','year_origine':'year'})
qog_knn_imputed_dest = qog_knn_imputed_dest.add_suffix('_dest').rename(columns={'ccodealp_dest':'ccodealp','year_dest':'year'})

In [59]:
df_finale = asylum.merge(qog_knn_imputed_origine,left_on=['citizen','TIME_PERIOD'],right_on=['ccodealp','year'],how='left').merge(qog_knn_imputed_dest,left_on=['citizen','TIME_PERIOD'],right_on=['ccodealp','year'],how='left')

In [60]:
df = df_finale.drop(['ccodealp_x','year_x','ccodealp_y','year_y'],axis=1)

In [76]:
df = df.dropna()

## Chômage

In [131]:
pays = list(list(df['citizen'].unique()) + list(df['geo'].unique()))
pays.remove('TWN')

In [133]:
unemp = pd.DataFrame(index=range(len(list(set(pays)))),columns=range(15))
unemp.columns = ['country','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']
unemp['country']= list(set(pays))
#unemp = unemp[~unemp['country'].isin(['TWN'])]

In [134]:
import datetime 
indicator_code = "SL.UEM.TOTL.ZS"
data_date = datetime.datetime(2008, 12, 31), datetime.datetime(2021, 12, 31) 

In [142]:
for i in range(len(unemp)):
    country_code = unemp['country'][i]
    for j in range(1,15):
        unemp.iloc[i,j] = wbdata.get_data(indicator=indicator_code, country=country_code, data_date=data_date)[j-1]['value']

In [147]:
unemp.iloc[74,1] = unemp.iloc[4,2]
unemp.iloc[157,1] = unemp.iloc[158,2]

In [148]:
supp = list(unemp[unemp['2008'].isna()]['country'])

In [150]:
df = df[~df['citizen'].isin(supp)]
df = df[~df['geo'].isin(supp)]

unemp = unemp[~unemp["2008"].isna()]

In [152]:
df_2008 = df[df['TIME_PERIOD']==2008]
df_2009 = df[df['TIME_PERIOD']==2009]
df_2010 = df[df['TIME_PERIOD']==2010]
df_2011 = df[df['TIME_PERIOD']==2011]
df_2012 = df[df['TIME_PERIOD']==2012]
df_2013 = df[df['TIME_PERIOD']==2013]
df_2014 = df[df['TIME_PERIOD']==2014]
df_2015 = df[df['TIME_PERIOD']==2015]
df_2016 = df[df['TIME_PERIOD']==2016]
df_2017 = df[df['TIME_PERIOD']==2017]
df_2018 = df[df['TIME_PERIOD']==2018]
df_2019 = df[df['TIME_PERIOD']==2019]
df_2020 = df[df['TIME_PERIOD']==2020]
df_2021 = df[df['TIME_PERIOD']==2021]

In [153]:
df_2008_bis = df_2008.merge(unemp[['country','2008']],left_on='citizen',right_on='country',how='left').rename(columns={'2008':'unemp_origine'}).merge(unemp[['country','2008']],left_on='geo',right_on='country',how='left').rename(columns={'2008':'unemp_dest'}).drop(['country_x','country_y'],axis=1)
df_2009_bis = df_2009.merge(unemp[['country','2009']],left_on='citizen',right_on='country',how='left').rename(columns={'2009':'unemp_origine'}).merge(unemp[['country','2009']],left_on='geo',right_on='country',how='left').rename(columns={'2009':'unemp_dest'}).drop(['country_x','country_y'],axis=1)
df_2010_bis = df_2010.merge(unemp[['country','2010']],left_on='citizen',right_on='country',how='left').rename(columns={'2010':'unemp_origine'}).merge(unemp[['country','2010']],left_on='geo',right_on='country',how='left').rename(columns={'2010':'unemp_dest'}).drop(['country_x','country_y'],axis=1)
df_2011_bis = df_2011.merge(unemp[['country','2011']],left_on='citizen',right_on='country',how='left').rename(columns={'2011':'unemp_origine'}).merge(unemp[['country','2011']],left_on='geo',right_on='country',how='left').rename(columns={'2011':'unemp_dest'}).drop(['country_x','country_y'],axis=1)
df_2012_bis = df_2012.merge(unemp[['country','2012']],left_on='citizen',right_on='country',how='left').rename(columns={'2012':'unemp_origine'}).merge(unemp[['country','2012']],left_on='geo',right_on='country',how='left').rename(columns={'2012':'unemp_dest'}).drop(['country_x','country_y'],axis=1)
df_2013_bis = df_2013.merge(unemp[['country','2013']],left_on='citizen',right_on='country',how='left').rename(columns={'2013':'unemp_origine'}).merge(unemp[['country','2013']],left_on='geo',right_on='country',how='left').rename(columns={'2013':'unemp_dest'}).drop(['country_x','country_y'],axis=1)
df_2014_bis = df_2014.merge(unemp[['country','2014']],left_on='citizen',right_on='country',how='left').rename(columns={'2014':'unemp_origine'}).merge(unemp[['country','2014']],left_on='geo',right_on='country',how='left').rename(columns={'2014':'unemp_dest'}).drop(['country_x','country_y'],axis=1)
df_2015_bis = df_2015.merge(unemp[['country','2015']],left_on='citizen',right_on='country',how='left').rename(columns={'2015':'unemp_origine'}).merge(unemp[['country','2015']],left_on='geo',right_on='country',how='left').rename(columns={'2015':'unemp_dest'}).drop(['country_x','country_y'],axis=1)
df_2016_bis = df_2016.merge(unemp[['country','2016']],left_on='citizen',right_on='country',how='left').rename(columns={'2016':'unemp_origine'}).merge(unemp[['country','2016']],left_on='geo',right_on='country',how='left').rename(columns={'2016':'unemp_dest'}).drop(['country_x','country_y'],axis=1)
df_2017_bis = df_2017.merge(unemp[['country','2017']],left_on='citizen',right_on='country',how='left').rename(columns={'2017':'unemp_origine'}).merge(unemp[['country','2017']],left_on='geo',right_on='country',how='left').rename(columns={'2017':'unemp_dest'}).drop(['country_x','country_y'],axis=1)
df_2018_bis = df_2018.merge(unemp[['country','2018']],left_on='citizen',right_on='country',how='left').rename(columns={'2018':'unemp_origine'}).merge(unemp[['country','2018']],left_on='geo',right_on='country',how='left').rename(columns={'2018':'unemp_dest'}).drop(['country_x','country_y'],axis=1)
df_2019_bis = df_2019.merge(unemp[['country','2019']],left_on='citizen',right_on='country',how='left').rename(columns={'2019':'unemp_origine'}).merge(unemp[['country','2019']],left_on='geo',right_on='country',how='left').rename(columns={'2019':'unemp_dest'}).drop(['country_x','country_y'],axis=1)
df_2020_bis = df_2020.merge(unemp[['country','2020']],left_on='citizen',right_on='country',how='left').rename(columns={'2020':'unemp_origine'}).merge(unemp[['country','2020']],left_on='geo',right_on='country',how='left').rename(columns={'2020':'unemp_dest'}).drop(['country_x','country_y'],axis=1)
df_2021_bis = df_2021.merge(unemp[['country','2021']],left_on='citizen',right_on='country',how='left').rename(columns={'2021':'unemp_origine'}).merge(unemp[['country','2021']],left_on='geo',right_on='country',how='left').rename(columns={'2021':'unemp_dest'}).drop(['country_x','country_y'],axis=1)

In [154]:
asylum = pd.concat([df_2008_bis,df_2009_bis,df_2010_bis,df_2011_bis,df_2012_bis,df_2013_bis,df_2014_bis,df_2015_bis,df_2016_bis,df_2017_bis,df_2018_bis,df_2019_bis,df_2020_bis,df_2021_bis])

In [ ]:
#pd.to_csv('asylum.csv')